In [18]:
import openai
import os
from tqdm import tqdm
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
def get_last_15_words(text):
    words = text.split()
    last_15_words = " ".join(words[-15:])
    return last_15_words

In [33]:
start_prompt = "Think about what research fields would be a good fit for you for PhD and why.\n"
thought = openai.Completion.create(model="davinci:ft-personal-2023-06-03-21-52-43", prompt=start_prompt, temperature=0.5, max_tokens=75)["choices"][0]["text"]
thoughts = [thought]

In [34]:
for _ in tqdm(range(15)):
    message=[
        {"role": "user", "content": """The following text enclosed in carrot brackets <> is a snippet of a stream of consciousness. Consider what the user is thinking about, and please produce a brief sentence written as an instruction that would cause the user to continue thinking, possibly with a slight change of topic to give direction to the thoughts. For example, your sentence could be formatted as "Think about ...", format your response as just this sentence and nothing else.\n\n<""" + thought + ">"}
    ]
    prompt = openai.ChatCompletion.create(model="gpt-4", messages=message)["choices"][0]["message"]["content"]
    prompt += " Here's where your thoughts left off, continue it': " + get_last_15_words(thought)
    thought = openai.Completion.create(model="davinci:ft-personal-2023-06-03-21-52-43", prompt=prompt, temperature=0.5, max_tokens=75)["choices"][0]["text"]
    thoughts.append(thought)

100%|██████████| 15/15 [00:58<00:00,  3.89s/it]


In [35]:
combined_string = " ".join(thoughts)
with open("results/output.txt", "w", encoding="utf-8") as file:
    file.truncate()
    file.write("Start Prompt: " + start_prompt + "\n")
    file.write(combined_string)